## Dask Dataframes

Dask possibilita que também sejam utilizados dataframes para processamento paralelo e distribuído.

Como exemplo, carregaremos arquivos referentes a registros de vôos no aeroporto de Nova York:

Para fazer o download da base:

```python
import urllib, tarfile
print('Realizando download...', end=' ')
url = 'https://storage.googleapis.com/dask-tutorial-data/nycflights.tar.gz'
filename, headers = urllib.request.urlretrieve(url, 'nycflights.tar.gz')
print('pronto.')

print('Descompactando...', end=' ')
with tarfile.open(filename, mode='r:gz') as flights:
    flights.extractall('base/')
print('pronto.')
!ls base/nycflights
```

Como comparação, carregaremos primeiramente a base utilizando Pandas:

```python
import pandas as pd
from glob import glob

df_pandas = pd.concat([pd.read_csv(file) for file in glob('base/nycflights/*.csv')])
```


Para abrir diversas bases, cada uma foi carregada em dataframe e armazenada em uma lista. Em seguida, estes dataframes foram concatenados, formando um único dataframe.

```python
df_pandas
```

Agora carregamos as bases utilizando Dask dataframes:

```python
import dask.dataframe as dd

df = dd.read_csv('base/nycflights/*.csv')
```

**Atividade**
- Inicialmente, quais foram as diferenças?
    - ao carregar a base, com `.read_csv()`
    - ao visualizar a base

Podemos visualizar mais informações sobre o dataframe:

```python
df.dask
```

Vamos observar também o DAG gerado:

```python
df.visualize()
```

Um Dask Dataframe é composto por um conjunto de Pandas Dataframes. 

<!-- ![Dask dataframes](https://docs.dask.org/en/stable/_images/dask-dataframe.svg) -->
<img src="https://docs.dask.org/en/stable/_images/dask-dataframe.svg"  width="30%" height="30%">

Imagem: [Dask DataFrame](https://docs.dask.org/en/stable/dataframe.html)

Vamos observar o tipo de dado que compõe a estrutura destes dataframes:

```python
type(df.partitions[0].compute())
```

E também a quantidade de dataframes:

```python
len(list(df.partitions))
```

Os arquivos ainda não foram carregados. Este passo será realizado ao executar o método `.compute()`.

Faremos isso em diferentes partes, observando alguns pontos.

```python
df.head()
```

Neste passo, observaremos um erro.

```python
df.tail()
```

Ao observar os erros:

```
ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+----------------+---------+----------+
| Column         | Found   | Expected |
+----------------+---------+----------+
| CRSElapsedTime | float64 | int64    |
| TailNum        | object  | float64  |
+----------------+---------+----------+
```

Este erro ocorreu porque o Dask estimou os tipos de dados com base nas primeiras linhas. Porém, haviam outras linhas da base em que o tipo de dado na coluna era diferente.

Para corrigir estes erros, devemos informar manualmente os tipos de dados de algumas colunas ao carregar a base.

```python
df = dd.read_csv('base/nycflights/*.csv', dtype={'TailNum': str, 'CRSElapsedTime':float})
df.tail()
```

## Realizando operações

Como exemplo, vamos tentar obter o maior valor presente em cada coluna, utilizando o método `.max()`.

```python
df_maiores = df.max()
```

Observando o DAG:
```python
maiores_valores.visualize()
```
ou então, apresentando com outro *layout*:
```python
df_maiores.visualize(rankdir='LR', size='12,12!')
```

Ou então
```python
df_maiores.visualize(rankdir='LR', size='12,12!')
```

Calculando os maiores valores
```python
df_maiores.compute()
```

Obtendo os maiores valores de uma única coluna:

```python
depDelay_max = df['DepDelay'].max()
```



```python
depDelay_max.visualize()
```

Compare com o DAG anterior (maior valor de todas as colunas):
```python
df.max().visualize()
```

## Referências

- [Dask. Scale the Python tools you love.](https://docs.dask.org/en/stable/)  
- [BOCHMAN, D. Dask: Machine Learning & Data Science Open-source Spotlight. Youtube.](https://www.youtube.com/watch?v=Alwgx_1qsj4&t=755s)
- [ROCKLIN, M. Dask DataFrame: An Introduction. Youtube](https://www.youtube.com/watch?v=AT2XtFehFSQ&t=262s)